## Setup

In [1]:
!git clone https://github.com/yeyinthtoon/tf2-resmlp.git
%cd tf2-resmlp/
!pip install .

Cloning into 'tf2-resmlp'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 50 (delta 16), reused 43 (delta 13), pack-reused 0
Unpacking objects: 100% (50/50), done.
/content/tf2-resmlp
Branch 'enchant-add-example' set up to track remote branch 'enchant-add-example' from 'origin'.
Switched to a new branch 'enchant-add-example'
Processing /content/tf2-resmlp
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel f

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from resmlp import ResMlp12

### TPU setup

In [3]:
try:
    tpu = (
        tf.distribute.cluster_resolver.TPUClusterResolver()
    )  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print("Running on TPU ", tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()
print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.49.133.194:8470
INFO:tensorflow:Initializing the TPU system: grpc://10.49.133.194:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.49.133.194:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


### Configs

In [4]:
configs = {
    "seed": 112,
    "data_augmentation": True,
    "epochs": 200,
    "batch_size": 256,
    "drop_path_rate": 0.3,
    "dropout_rate": 0.5,
    "num_classes": 10,
    "input_image_size": [224, 224],
}

## Dataset Preparation

In [5]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

x_train_mean = np.mean(x_train, axis=0)
x_train -= x_train_mean
x_test -= x_train_mean

170500096/170498071 [==============================] - 2s 0us/step


In [6]:
def augment(image, label):
    image = tf.image.random_flip_left_right(image)
    return image, label


def set_shape(image, label):
    image = tf.image.resize(image, configs["input_image_size"])
    image.set_shape(configs["input_image_size"] + [3])
    label.set_shape(
        [
            1,
        ]
    )
    return image, label


train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = train_ds.shuffle(10000, seed=configs["seed"])
if configs["data_augmentation"]:
    train_ds = train_ds.map(augment)
train_ds = train_ds.map(set_shape)
train_ds = train_ds.batch(configs["batch_size"], drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)


test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_ds = test_ds.map(set_shape)
test_ds = test_ds.batch(configs["batch_size"], drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)

## Training

In [7]:
tf.keras.backend.clear_session()
with strategy.scope():
    model = ResMlp12(
        input_shape=configs["input_image_size"] + [3],
        num_classes=configs["num_classes"],
        dropout_rate=configs["dropout_rate"],
        drop_path_rate=configs["drop_path_rate"],
    )
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
    model.compile(
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer="adam",
        metrics=["accuracy"],
    )

In [8]:
model.summary()

Model: "resmlp12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
patch_embedding_conv_1 (Conv2D) (None, 14, 14, 384)  295296      input_1[0][0]                    
__________________________________________________________________________________________________
rearrange (Rearrange)           (None, 196, 384)     0           patch_embedding_conv_1[0][0]     
__________________________________________________________________________________________________
block_0_affine_1 (DiagonalAffin (None, 196, 384)     768         rearrange[0][0]                  
___________________________________________________________________________________________

In [9]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print("Learning rate: ", lr)
    return lr


lr_scheduler = keras.callbacks.LearningRateScheduler(lr_schedule)

In [10]:
history = model.fit(
    train_ds,
    epochs=configs["epochs"],
    validation_data=(test_ds),
    callbacks=[lr_scheduler],
)

Epoch 1/200
Learning rate:  0.001
195/195 [==============================] - 59s 120ms/step - loss: 1.8074 - accuracy: 0.3406 - val_loss: 1.4649 - val_accuracy: 0.4724
Epoch 2/200
Learning rate:  0.001
195/195 [==============================] - 17s 88ms/step - loss: 1.4310 - accuracy: 0.4870 - val_loss: 1.2728 - val_accuracy: 0.5303
Epoch 3/200
Learning rate:  0.001
195/195 [==============================] - 17s 88ms/step - loss: 1.2669 - accuracy: 0.5476 - val_loss: 1.1454 - val_accuracy: 0.5817
Epoch 4/200
Learning rate:  0.001
195/195 [==============================] - 17s 88ms/step - loss: 1.1619 - accuracy: 0.5846 - val_loss: 1.0282 - val_accuracy: 0.6346
Epoch 5/200
Learning rate:  0.001
195/195 [==============================] - 17s 88ms/step - loss: 1.0979 - accuracy: 0.6078 - val_loss: 1.0055 - val_accuracy: 0.6386
Epoch 6/200
Learning rate:  0.001
195/195 [==============================] - 17s 89ms/step - loss: 1.0430 - accuracy: 0.6290 - val_loss: 0.9465 - val_accuracy: 0.65